# <font color=green>*BCBIO 490*</font>

In [223]:
import numpy as np
import os
import math
from Bio import SeqIO

class Sequence_Alignment:
    
    def __init__(self, match=10, mismatch=-20, gapOpen=40, gapExtend=2, diff_block=200, ambiguous=0):
        self.gapOpen = abs(gapOpen)
        self.gapExtend = abs(gapExtend)
        self.diff = abs(diff_block)
        self.sub_mat = np.zeros((129,129))
        for i in range(len(self.sub_mat)):
            for j in range(len(self.sub_mat[i])):
                if i == 'N' or j == 'N':
                    self.sub_mat[i][j] = 0 
                elif i != j:
                    self.sub_mat[i][j] = mismatch 
                else:
                    self.sub_mat[i][j] = match
    
    def getScore(self, nuc1, nuc2):
        return self.sub_mat[ord(nuc1.upper())][ord(nuc2.upper())]
    
    def setGapOpenPenalty(self, penalty):
        self.gapOpen = abs(penalty)
        
    def setGapExtensionPenalty(self, penalty):
        self.gapExtend = abs(penalty)
        
    def setDifferenceBlockPenalty(self, penalty):
        self.diff = abs(penalty)
        
    def alignInput(self, fasta1, fasta2, align_type):
        """
        cwd = os.getcwd()  # Get the current working directory (cwd)
        files = os.listdir(cwd)  # Get all the files in that directory
        print("Files in %r: %s" % (cwd, files))
        """
        if os.path.getsize(fasta1) == 0 or os.path.getsize(fasta2) == 0:
            return "Input contains empty file(s)"
        if align_type != 'global' and align_type != 'local':
            return "This program only has two alignment types: 'global' or 'local'"
                    
        with open(fasta1) as handle1, open(fasta2) as handle2:
            for record1, record2 in zip(SeqIO.parse(handle1, "fasta"), SeqIO.parse(handle2, "fasta")):
                print("Aligning " + record1.id + " with " + record2.id + " using " + align_type + " alignment: ")
                print()
                if align_type == 'global':
                    self.global_alignment(record1.seq, record2.seq)
                elif align_type == 'local':
                    self.local_alignment(record1.seq, record2.seq)

                    
    def global_alignment_trace(self, seq1, seq2):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        i_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] = i_mat[len(seq1)][i + 1] - self.gapExtend
            s_mat[len(seq1)][i] = i_mat[len(seq1)][i]
            d_mat[len(seq1)][i] = s_mat[len(seq1)][i] - self.gapOpen
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] = d_mat[j + 1][len(seq2)] - self.gapExtend
            s_mat[j][len(seq2)] = d_mat[j][len(seq2)]
            i_mat[j][len(seq2)] = s_mat[j][len(seq2)] - self.gapOpen
            
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k]), d_mat[j][k], i_mat[j][k])
                
        current = 'S'
        i = j = 0
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) and j == len(seq2):
                    break
                elif i == len(seq1) or s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif j == len(seq2) or s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
                
            # Add another statement for H matrix, h(i,j) = max{h(i+1,j), h(i,j+1)}. 
            # First is deletion gap, latter is insertion gap.
        
        return [s_mat[0][0], align_seq1, align_mid, align_seq2]
    
    def global_alignment(self, seq1, seq2):
        arr = self.global_alignment_trace(seq1, seq2)
        self.printAlignment(seq1, seq2, arr[0], arr[1], arr[2], arr[3])
    
    # Add information about number of similarity blocks and their corresponding percent identity. 
    def printAlignment(self, seq1, seq2, score, align_seq1, align_mid, align_seq2):
        counter = 0
        curr1 = 1
        curr2 = 1
        max_space = len(str(max(len(seq1), len(seq2))))
        print("Alignment Score: " + str(score))
        print("Sequences Similarity: " + str(round((align_mid.count("|")/ len(align_mid)) * 100,2)) + "%\n")
        while counter <= len(align_seq1):
            print("Sequence 1 > " + self.generateString('left', max_space, str(curr1)) + align_seq1[counter: counter + 79] + self.generateString('right', max_space, str(min(curr1 + 79 - align_seq1[counter: counter + 79].count('-') - 1, len(seq1)))))
            print("             " + self.generateString('left', max_space, "") + align_mid[counter: counter + 79] + self.generateString('right', max_space, ""))
            print("Sequence 2 > " + self.generateString('left', max_space, str(curr2)) + align_seq2[counter: counter + 79] + self.generateString('right', max_space, str(min(curr2 + 79 - align_seq2[counter: counter + 79].count('-') - 1, len(seq2)))))
            
            print("\n")
            curr1 = min(curr1 + 79 - align_seq1[counter: counter + 79].count('-'), len(seq1))
            curr2 = min(curr2 + 79 - align_seq2[counter: counter + 79].count('-'), len(seq2))
            counter += 79
    
    def local_alignment(self, seq1, seq2):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        highestScore = 0
        firstRow = len(seq1)
        firstCol = len(seq2)
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        i_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
            d_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
            i_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
        
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(0, s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k]), d_mat[j][k], i_mat[j][k])
                
                if highestScore < s_mat[j][k]:
                    highestScore = s_mat[j][k]
                    firstRow = j
                    firstCol = k
                    
        current = 'S'
        i = firstRow
        j = firstCol 
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) or j == len(seq2) or s_mat[i][j] == 0:
                    break
                elif s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue 
                
        lastRow = i
        lastCol = j
        counter = 0
        curr1 = firstRow + 1
        curr2 = firstCol + 1
        max_space = len(str(max(len(seq1), len(seq2))))
        print("Alignment Score: " + str(s_mat[firstRow][firstCol]) + "\n")
        while counter <= len(align_seq1):
            print("Sequence 1 > " + self.generateString('left', max_space, str(curr1)) + align_seq1[counter: counter + 79] + self.generateString('right', max_space, str(min(curr1 + 79 - align_seq1[counter: counter + 79].count('-') - 1, lastRow))))
            print("             " + self.generateString('left', max_space, "") + align_mid[counter: counter + 79] + self.generateString('right', max_space, ""))
            print("Sequence 2 > " + self.generateString('left', max_space, str(curr2)) + align_seq2[counter: counter + 79] + self.generateString('right', max_space, str(min(curr2 + 79 - align_seq2[counter: counter + 79].count('-') - 1, lastCol))))
            print("\n")
            curr1 = min(curr1 + 79 - align_seq1[counter: counter + 79].count('-'), lastRow)
            curr2 = min(curr2 + 79 - align_seq2[counter: counter + 79].count('-'), lastCol)
            counter += 79
            
    def gap3_traceback(self, seq1, seq2):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        h_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        i_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        h_mat[len(seq1)][len(seq2)] = -self.diff
        
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] = i_mat[len(seq1)][i + 1] - self.gapExtend
            h_mat[len(seq1)][i] = -self.diff
            s_mat[len(seq1)][i] = max(i_mat[len(seq1)][i], h_mat[len(seq1)][i])
            d_mat[len(seq1)][i] = s_mat[len(seq1)][i] - self.gapOpen
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] = d_mat[j + 1][len(seq2)] - self.gapExtend
            h_mat[j][len(seq2)] = -self.diff
            s_mat[j][len(seq2)] = max(d_mat[j][len(seq2)], h_mat[j][len(seq2)])
            i_mat[j][len(seq2)] = s_mat[j][len(seq2)] - self.gapOpen
            
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                h_mat[j][k] = max(h_mat[j + 1][k], h_mat[j][k + 1], s_mat[j + 1][k] - self.diff, s_mat[j][k + 1] - self.diff)
                s_mat[j][k] = max(s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k]), d_mat[j][k], i_mat[j][k], h_mat[j][k])
                
        current = 'S'
        i = j = 0
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) and j == len(seq2):
                    break
                elif s_mat[i][j] == h_mat[i][j]:
                    current = 'H'
                    continue
                elif i == len(seq1) or s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif j == len(seq2) or s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                    
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
                
            elif current == 'H':
                if i + 1 <= len(seq1):
                    if h_mat[i][j] == h_mat[i + 1][j] or h_mat[i][j] == s_mat[i + 1][j] - self.diff:
                        align_seq1 += seq1[i]
                        align_seq2 += " "
                        align_mid += "+"
                        if h_mat[i][j] == s_mat[i + 1][j] - self.diff:
                            current = 'S'
                        i += 1
                        continue
                        
                if j + 1 <= len(seq2):
                    if h_mat[i][j] == h_mat[i][j + 1] or h_mat[i][j] == s_mat[i][j + 1] - self.diff:
                        align_seq1 += " "
                        align_seq2 += seq2[j]
                        align_mid += "+"
                        if h_mat[i][j] == s_mat[i][j + 1] - self.diff:
                            current = 'S'
                        j += 1
                        continue

        return [s_mat[0][0], align_seq1, align_mid, align_seq2]
    
    def gap3_trace(self, seq1, seq2, s, d, h, S, D, H):
        if len(seq1) <= 50:
            arr = self.gap3_traceback(seq1, seq2)
            return arr
        else: 
            M = len(seq1)
            N = len(seq2)
            imid = M // 2
        
            s_row = np.zeros(N + 1)
            i_row = np.zeros(N + 1)
            d_row = np.zeros(N + 1)
            h_row = np.zeros(N + 1)
        
            S_ROW = np.zeros(N + 1)
            I_ROW = np.zeros(N + 1)
            D_ROW = np.zeros(N + 1)
            H_ROW = np.zeros(N + 1)
        
            s_row[0] = s
            i_row[0] = s_row[0] - self.gapOpen
            d_row[0] = d
            h_row[0] = h
        
            S_ROW[N] = S
            I_ROW[N] = S_ROW[N] - self.gapOpen
            H_ROW[N] = H
            D_ROW[N] = D
        
            for j in range(1, N + 1):
                i_row[j] = i_row[j-1] - self.gapExtend
                h_row[j] = -self.diff
                s_row[j] = max(i_row[j], h_row[j])
                d_row[j] = s_row[j] - self.gapOpen
            
            for i in range(1, imid + 1):
                s_prev = s_row[0]
            
                d_row[0] = d_row[0] - self.gapExtend
                h_row[0] = -self.diff
                s_row[0] = max(d_row[0], h_row[0])
                i_row[0] = s_row[0] - self.gapOpen
            
                for j in range(1, N + 1):
                    d_row[j] = max(d_row[j] - self.gapExtend, s_row[j] - self.gapOpen - self.gapExtend)
                    i_row[j] = max(i_row[j - 1] - self.gapExtend, s_row[j - 1] - self.gapOpen - self.gapExtend)
                    h_row[j] = max(h_row[j - 1], h_row[j], s_row[j - 1] - self.diff, s_row[j] - self.diff)
                    tmp = s_row[j]
                    s_row[j] = max(s_prev + self.getScore(seq1[i-1], seq2[j-1]), d_row[j], i_row[j], h_row[j])
                    s_prev = tmp

            for j in range(N - 1, -1, -1):
                I_ROW[j] = I_ROW[j + 1] - self.gapExtend
                H_ROW[j] = -self.diff
                S_ROW[j] = max(I_ROW[j], H_ROW[j])
                D_ROW[j] = S_ROW[j] - self.gapOpen
            
            for i in range(M - 1, imid - 1, -1):
                S_PREV = S_ROW[N]
            
                D_ROW[N] = D_ROW[N] - self.gapExtend
                H_ROW[N] = -self.diff
                S_ROW[N] = max(D_ROW[N], H_ROW[N])
                I_ROW[N] = S_ROW[N] - self.gapOpen
            
                for j in range(N - 1, -1, -1):
                    D_ROW[j] = max(D_ROW[j] - self.gapExtend, S_ROW[j] - self.gapOpen - self.gapExtend)
                    I_ROW[j] = max(I_ROW[j + 1] - self.gapExtend, S_ROW[j + 1] - self.gapOpen - self.gapExtend)
                    H_ROW[j] = max(H_ROW[j + 1], H_ROW[j], S_ROW[j + 1] - self.diff, S_ROW[j] - self.diff)
                    tmp = S_ROW[j]
                    S_ROW[j] = max(S_PREV + self.getScore(seq1[i], seq2[j]), D_ROW[j], I_ROW[j], H_ROW[j])
                    S_PREV = tmp
        
            hk = -math.inf
            df = -math.inf
            st = -math.inf
        
            jh = 0
            jd = 0
            js = 0
        
            for j in range(N + 1):
                if h_row[j] + H_ROW[j] + self.diff > hk:
                    hk = h_row[j] + H_ROW[j] + self.diff
                    jh = j
                if d_row[j] + D_ROW[j] + self.gapOpen > df:
                    df = d_row[j] + D_ROW[j] + self.gapOpen
                    jd = j
                if s_row[j] + S_ROW[j] > st:
                    st = s_row[j] + S_ROW[j]
                    js = j

            if max(hk, df, st) == st:
                jmid = js
            
            elif max(hk, df, st) == df:
                jmid = jd
            
            else:
                jmid = jh 
        
            top_align = self.gap3_trace(seq1[0: imid + 1], seq2[0:jmid + 1], s, d, h, 
                                         S_ROW[jmid], D_ROW[jmid], H_ROW[jmid])
        
            bottom_align = self.gap3_trace(seq1[imid+1::], seq2[jmid+1::], s_row[jmid], 
                                            d_row[jmid], h_row[jmid], S, D, H)
        
            final_score = top_align[0] + bottom_align[0]
            final_align_seq1 = top_align[1] + bottom_align[1]
            final_align_mid = top_align[2] + bottom_align[2]
            final_align_seq2 = top_align[3] + bottom_align[3]
        
            return [final_score, final_align_seq1, final_align_mid, final_align_seq2]
        
        
    def gap3(self, seq1, seq2):
        s = 0
        d = -self.gapOpen
        h = -self.diff
        arr = self.gap3_trace(seq1, seq2, s, d, h, s, d, h)
        self.printAlignment(seq1, seq2, arr[0], arr[1], arr[2], arr[3])
        
    def generateString(self, position, max_space, string):
        if len(string) > max_space:
            return
        if position == 'left':
            return string + ((max_space - len(string)) * " ") + " "
        elif position == 'right':
            return " " + ((max_space - len(string)) * " ") + string
        else:
            return


In [221]:
p = Sequence_Alignment()
A = "ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGATGCTAGCTAGCTTAAAATAAAAAAAAAAAAAAAAAAA"
B = "TAGACACGTAGCTGATCGATGTACGATCGATCGAGATCGATCGATCGATCGCTAACTGATGCTAGCTAGCTTAT"
print(len(A))
print(len(B))
p.gap3(A, B)
p.global_alignment(A, B)

100
74
Alignment Score: 438.0
Sequences Similarity: 71.0%

Sequence 1 > 1   ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGAT  64
                  |*|||| ||||||||||||||||||||||||||||  ||||||||||||||||||||*|||||    
Sequence 2 > 1   -TAGACA-CGTAGCTGATCGATGTACGATCGATCGA--GATCGATCGATCGATCGCTAACTGAT  60


Sequence 1 > 80  ATAAAAAAAAAAAAAAAAAAA 100
                                    |*    
Sequence 2 > 73  -------------------AT  74


Alignment Score: 438.0
Sequences Similarity: 71.0%

Sequence 1 > 1   ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGAT  64
                  |*|||| ||||||||||||||||||||||||||||  ||||||||||||||||||||*|||||    
Sequence 2 > 1   -TAGACA-CGTAGCTGATCGATGTACGATCGATCGA--GATCGATCGATCGATCGCTAACTGAT  60


Sequence 1 > 80  ATAAAAAAAAAAAAAAAAAAA 100
                                    |*    
Sequence 2 > 73  -------------------AT  74




In [228]:
p = Sequence_Alignment(mismatch=-10, gapOpen=40, gapExtend=2, diff_block=20)
A = "GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGGGCAGTAGCCTTCCC"
A+="CAGAGCCCCCTAGCCGCAGGCACCAGAGGGTCCAAGACAAGACTGGAAGGGCACCTCGGGTTCGGGAGGAGCTGTGAGTGGCT"
B = "GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGGGGGCCGGCCAGGGTGCTACCCCAAGCCTACTGACTGTCTTACTGG"
B+= "CAAGCTTCAGCGAGTCCAGGAGAAAGCTGGGAAGCCCCGCCGGGTCCGGGTCCGAGAGGAACTGTGAATGGCTGAGCCTGCTTCTCGAGGATCAGGC"
print(len(A))
print(len(B))
p.gap3(A,B)

176
186
Alignment Score: 760.0
Sequences Similarity: 47.88%

Sequence 1 > 1   GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGG  79
                 +++++++++++++++++++++++++|||||||*|||||||||||||||||||||||*|||||||||*||+++|||||||    
Sequence 2 > 1                            GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGG   GGGCCGG  79


Sequence 1 > 80  GCAGTAGC    CTTCCCCAGAGCCCCCTAGCCGCA              GGCAC     CAGAGGGTCCAAGACAAGA 158
                 *|||++++++++||*|||||+||+++||||++++++++++++++++++++||||++++++|||*|*|||||*||*||++    
Sequence 2 > 80  CCAG    GGTGCTACCCCA AG   CCTA      CTGACTGTCTTACTGGCA AGCTTCAGCGAGTCCAGGAGAA   158


Sequence 1 > 159 CT     GGAAGGGCACCT  CGGGTTCGG      GAGGAGCTGTGAGTGGCT                         176
                 +++++++||||++||*||+++|||||*|||++++++|||||*||||||*|||||++++++++++++++++++++++++    
Sequence 2 > 159   AGCTGGGAA  GCCCC GCCGGGTCCGGGTCCGAGAGGAACTGTGAATGGCTGAGCCTGCTTCTCGAGGATCAGGC 186




In [229]:
p = Sequence_Alignment(gapOpen=40, gapExtend=2, diff_block=20)
A = "AACCCCAAAA"
B = "AAA"
p.gap3(A,B)

Alignment Score: 10.0
Sequences Similarity: 30.0%

Sequence 1 > 1  AACCCCAAAA 10
                +++++++|||   
Sequence 2 > 1         AAA  3


